**Packages**

In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import h5py
import math

In [2]:
EXTRACT_FREQUENCY = 15

In [3]:
!pip install ffmpeg --upgrade

## KTS Shot Boundary Detection

##### )) Paths

In [4]:
preprocessed_path='../../Preprocessing'
datasets_path='../../data'
public_dataset_path=datasets_path+'/Public datasets'
tvsum_data = public_dataset_path+'/ydata-tvsum50-v1_1'
# summe_data = public_dataset_path+'/SUMMe'
# custom_data = datasets_path+'/Custom data'


features_path = preprocessed_path+'/extracted_features'
normalFt_path = features_path+'/normal'
hashbasedFt_path = features_path+'/hashbased'
prebuiltFt_path = features_path+'/Prebuilt'

##### )) KTS Modules

In [5]:

def calc_scatters(K):
    """
    Calculate scatter matrix:
    scatters[i,j] = {scatter of the sequence with starting frame i and ending frame j}
    """
    n = K.shape[0]
    K1 = np.cumsum([0] + list(np.diag(K)))
    K2 = np.zeros((n+1, n+1))
    K2[1:, 1:] = np.cumsum(np.cumsum(K, 0), 1); # TODO: use the fact that K - symmetric

    scatters = np.zeros((n, n))

    diagK2 = np.diag(K2)

    i = np.arange(n).reshape((-1,1))
    j = np.arange(n).reshape((1,-1))
    scatters = (K1[1:].reshape((1,-1))-K1[:-1].reshape((-1,1))
                - (diagK2[1:].reshape((1,-1)) + diagK2[:-1].reshape((-1,1)) - K2[1:,:-1].T - K2[:-1,1:]) / ((j-i+1).astype(float) + (j==i-1).astype(float)))
    scatters[j<i]=0

    return scatters

def cpd_nonlin(K, ncp, lmin=1, lmax=100000, backtrack=True, verbose=True,
    out_scatters=None):
    """ Change point detection with dynamic programming
    K - square kernel matrix
    ncp - number of change points to detect (ncp >= 0)
    lmin - minimal length of a segment
    lmax - maximal length of a segment
    backtrack - when False - only evaluate objective scores (to save memory)
    Returns: (cps, obj)
        cps - detected array of change points: mean is thought to be constant on [ cps[i], cps[i+1] )
        obj_vals - values of the objective function for 0..m changepoints
    """
    m = int(ncp)  # prevent numpy.int64

    (n, n1) = K.shape
    assert(n == n1), "Kernel matrix awaited."

    assert(n >= (m + 1)*lmin)
    assert(n <= (m + 1)*lmax)
    assert(lmax >= lmin >= 1)

    if verbose:
        #print "n =", n
        print("Precomputing scatters...")
    J = calc_scatters(K)

    if out_scatters != None:
        out_scatters[0] = J

    if verbose:
        print("Inferring best change points...")
    # I[k, l] - value of the objective for k change-points and l first frames
    I = 1e101*np.ones((m+1, n+1))
    I[0, lmin:lmax] = J[0, lmin-1:lmax-1]

    if backtrack:
        # p[k, l] --- "previous change" --- best t[k] when t[k+1] equals l
        p = np.zeros((m+1, n+1), dtype=int)
    else:
        p = np.zeros((1,1), dtype=int)

    for k in range(1,m+1):
        for l in range((k+1)*lmin, n+1):
            tmin = max(k*lmin, l-lmax)
            tmax = l-lmin+1
            c = J[tmin:tmax,l-1].reshape(-1) + I[k-1, tmin:tmax].reshape(-1)
            I[k,l] = np.min(c)
            if backtrack:
                p[k,l] = np.argmin(c)+tmin

     # Collect change points
    cps = np.zeros(m, dtype=int)

    if backtrack:
        cur = n
        for k in range(m, 0, -1):
            cps[k-1] = p[k, cur]
            cur = cps[k-1]

    scores = I[:, n].copy()
    scores[scores > 1e99] = np.inf
    return cps, scores

In [6]:
import numpy as np
def cpd_auto(K, ncp, vmax, desc_rate=1, **kwargs):
    """Main interface
    
    Detect change points automatically selecting their number
        K       - kernel between each pair of frames in video
        ncp     - maximum ncp
        vmax    - special parameter
    Optional arguments:
        lmin     - minimum segment length
        lmax     - maximum segment length
        desc_rate - rate of descriptor sampling (vmax always corresponds to 1x)
    Note:
        - cps are always calculated in subsampled coordinates irrespective to
            desc_rate
        - lmin and m should be in agreement
    ---
    Returns: (cps, costs)
        cps   - best selected change-points
        costs - costs for 0,1,2,...,m change-points
        
    Memory requirement: ~ (3*N*N + N*ncp)*4 bytes ~= 16 * N^2 bytes
    That is 1,6 Gb for the N=10000.
    """
    m = ncp
    (cpsx, scores) = cpd_nonlin(K, m, backtrack=False, **kwargs)
    
    N = K.shape[0]
    N2 = N*desc_rate  # length of the video before subsampling
    
    penalties = np.zeros(m+1)
    # Prevent division by zero (in case of 0 changes)
    ncp = np.arange(1, m+1)
    penalties[1:] = (vmax*ncp/(2.0*N2))*(np.log(float(N2)/ncp)+1)
    
    costs = scores/float(N) + penalties
    m_best = np.argmin(costs)
    (cps, scores2) = cpd_nonlin(K, m_best, **kwargs)

    return (cps, costs)
    

# ------------------------------------------------------------------------------
# Extra functions (currently not used)

def estimate_vmax(K_stable):
    """K_stable - kernel between all frames of a stable segment"""
    n = K_stable.shape[0]
    vmax = np.trace(centering(K_stable)/n)
    return vmax


def centering(K):
    """Apply kernel centering"""
    mean_rows = np.mean(K, 1)[:, np.newaxis]
    return K - mean_rows - mean_rows.T + np.mean(mean_rows)


def eval_score(K, cps):
    """ Evaluate unnormalized empirical score
        (sum of kernelized scatters) for the given change-points """
    N = K.shape[0]
    cps = [0] + list(cps) + [N]
    V1 = 0
    V2 = 0
    for i in range(len(cps)-1):
        K_sub = K[cps[i]:cps[i+1], :][:, cps[i]:cps[i+1]]
        V1 += np.sum(np.diag(K_sub))
        V2 += np.sum(K_sub) / float(cps[i+1] - cps[i])
    return (V1 - V2)


def eval_cost(K, cps, score, vmax):
    """ Evaluate cost function for automatic number of change points selection
    K      - kernel between all frames
    cps    - selected change-points
    score  - unnormalized empirical score (sum of kernelized scatters)
    vmax   - vmax parameter"""
    
    N = K.shape[0]
    penalty = (vmax*len(cps)/(2.0*N))*(np.log(float(N)/len(cps))+1)
    return score/float(N) + penalty

##### )) Change Points Generator

In [2]:
def gen_cps(H5_FILE):
    with h5py.File(H5_FILE,'a') as d:
        for key in d.keys():
            
            X = d[key+'/features'][()]
            
            n_frames = d[key + '/n_frames'][()]
            # n = X.shape[0]
            # n1 = min(n, 338) # 95%
            # m = round(n_frames/106*2)  ##ncp
            
            n = n_frames /  d[key + '/fps'][()]
            m = int(math.ceil(n/2.0))
            # m=50
            K1 = np.dot(X, X.T)
            print(m)
            # print('i was here.', K1.shape)
            cps1, scores1 = cpd_auto(K1, m, 1)

            cps1 *= EXTRACT_FREQUENCY
            cps1 = np.hstack((0, cps1, n_frames))
            
            begin_frames = cps1[:-1]
            end_frames = cps1[1:]
            cps1 = np.vstack((begin_frames, end_frames - 1)).T
            d.create_dataset(key+'/change_points', data=cps1)
            print(key,  n, m, cps1.shape)
            n_frame_per_seg = end_frames - begin_frames
            d.create_dataset(key+'/n_frame_per_seg',data=n_frame_per_seg)

#### )) TVSum Segmentation

In [11]:
# h5file_path= normalFt_path+'/TVSum.h5' 

# gen_cps(h5file_path)

183
Precomputing scatters...
Inferring best change points...
Precomputing scatters...
Inferring best change points...
video_1 365.41379310344826 183 (184, 2)
69
Precomputing scatters...
Inferring best change points...
Precomputing scatters...
Inferring best change points...
video_10 137.75862068965517 69 (70, 2)
82
Precomputing scatters...
Inferring best change points...
Precomputing scatters...
Inferring best change points...
video_11 162.06896551724137 82 (83, 2)
233
Precomputing scatters...
Inferring best change points...
Precomputing scatters...
Inferring best change points...
video_12 465.8965517241379 233 (234, 2)
71
Precomputing scatters...
Inferring best change points...
Precomputing scatters...
Inferring best change points...
video_13 141.28 71 (72, 2)
98
Precomputing scatters...
Inferring best change points...
Precomputing scatters...
Inferring best change points...
video_14 194.12 98 (99, 2)
73
Precomputing scatters...
Inferring best change points...
Precomputing scatters...

In [28]:
# h5file_path= prebuiltFt_path+'/eccv16_dataset_tvsum_google_pool5.h5' 

# gen_cps(h5file_path)

130
Precomputing scatters...
Inferring best change points...
Precomputing scatters...
Inferring best change points...
video_50 461 130 (103, 2) [[   0  119]
 [ 120  179]
 [ 180  299]
 [ 300  359]
 [ 360  419]
 [ 420  434]
 [ 435  449]
 [ 450  539]
 [ 540  599]
 [ 600  719]
 [ 720  734]
 [ 735  809]
 [ 810  839]
 [ 840  899]
 [ 900  914]
 [ 915  929]
 [ 930  944]
 [ 945  974]
 [ 975 1049]
 [1050 1169]
 [1170 1274]
 [1275 1394]
 [1395 1529]
 [1530 1619]
 [1620 1649]
 [1650 1724]
 [1725 1754]
 [1755 1829]
 [1830 1889]
 [1890 1934]
 [1935 2009]
 [2010 2294]
 [2295 2369]
 [2370 2384]
 [2385 2444]
 [2445 2489]
 [2490 2549]
 [2550 2639]
 [2640 2924]
 [2925 2999]
 [3000 3044]
 [3045 3134]
 [3135 3239]
 [3240 3269]
 [3270 3284]
 [3285 3404]
 [3405 3464]
 [3465 3494]
 [3495 3539]
 [3540 3599]
 [3600 3614]
 [3615 3674]
 [3675 3704]
 [3705 3749]
 [3750 3764]
 [3765 3809]
 [3810 3854]
 [3855 4004]
 [4005 4064]
 [4065 4124]
 [4125 4184]
 [4185 4304]
 [4305 4364]
 [4365 4409]
 [4410 4439]
 [4440 4604

------------------------

## Testing segmentation

**Creating Subshots**

In [51]:
from moviepy.tools import subprocess_call
from moviepy.config import get_setting

def ffmpeg_extract_subclip(filename, framepicks):
    """ Makes a new video file playing video file ``filename`` between
    the times ``t1`` and ``t2``. """
    name, ext = os.path.splitext(filename)
    cap = cv2.VideoCapture(public_dataset_path+"/ydata-tvsum50-v1_1/video/0tmA_C6XwfM.mp4")
    fps = cap.get(cv2.CAP_PROP_FPS)
    print('fps:', fps)
    cap.release()
    c=0
    for T1, T2 in framepicks:
        
        t1=T1/fps
        t2=T2/fps
        targetname = "%sSUB%d_%d.%s" % (shots_folder+'shot_'+str(c), t1, t2, ext)
        c+=1

        cmd = [get_setting("FFMPEG_BINARY"),"-y",
               "-ss", str(t1),
               "-i", filename,
               "-t", str(t2-t1),
               "-vcodec", "copy", "-acodec", "copy", targetname]
        
        print(t1, t2)
        subprocess_call(cmd)

------------------------